In [3]:
import cv2
from ultralytics import YOLO
import os
import numpy as np

# YOLO 모델 로드
model = YOLO('best (1).pt')  # YOLOv8 모델 경로로 변경

# 현재 폴더의 모든 JPG 파일 검색
image_files = [f for f in os.listdir() if f.endswith('.jpg')]

# 얼굴 인식 결과 시각화 함수
def draw_boxes(results, img):
    for result in results:
        boxes = result.boxes.xyxy  # x1, y1, x2, y2 format
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    return img

# 얼굴 원형 모자이크 처리 함수
def blur_faces(results, img):
    for result in results:
        boxes = result.boxes.xyxy  # x1, y1, x2, y2 format
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            radius = max((x2 - x1) // 2, (y2 - y1) // 2)
            mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
            cv2.circle(mask, (center_x, center_y), radius, (255, 255, 255), -1)
            face = cv2.GaussianBlur(img, (99, 99), 30)
            img = np.where(mask[:, :, None] == 255, face, img)
    return img

# 이미지 처리 함수
def process_image(img_path):
    # 이미지 읽기
    original_img = cv2.imread(img_path)
    result_img = original_img.copy()

    # 추론
    results = model(img_path, conf=0.25, imgsz=1280, max_det=1000)

    # 얼굴 인식 결과 시각화
    detected_img = draw_boxes(results, original_img.copy())

    # 얼굴 원형 모자이크 처리
    blurred_img = blur_faces(results, result_img)

    # 결과 이미지 저장 경로 설정
    detected_path = img_path.replace('.jpg', '_detected.jpg')
    blurred_path = img_path.replace('.jpg', '_result_blurred.jpg')

    # 결과 이미지 저장
    cv2.imwrite(detected_path, detected_img)
    cv2.imwrite(blurred_path, blurred_img)

    # 결과 이미지를 시스템 기본 이미지 뷰어로 열기
    if os.name == 'posix':  # MacOS
        os.system(f'open {detected_path}')
        os.system(f'open {blurred_path}')
    elif os.name == 'nt':  # Windows
        os.system(f'start {detected_path}')
        os.system(f'start {blurred_path}')
    else:  # Linux
        os.system(f'xdg-open {detected_path}')
        os.system(f'xdg-open {blurred_path}')

    print(f"{img_path} 이미지 처리가 완료되었습니다.")

# 모든 JPG 파일 처리
for img_file in image_files:
    process_image(img_file)



image 1/1 /Users/gw/Desktop/python/face_mosaic_project/test1.jpg: 1280x960 7 faces, 420.5ms
Speed: 12.6ms preprocess, 420.5ms inference, 0.7ms postprocess per image at shape (1, 3, 1280, 960)
test1.jpg 이미지 처리가 완료되었습니다.

image 1/1 /Users/gw/Desktop/python/face_mosaic_project/test2.jpg: 960x1280 5 faces, 454.0ms
Speed: 9.2ms preprocess, 454.0ms inference, 1.4ms postprocess per image at shape (1, 3, 960, 1280)
test2.jpg 이미지 처리가 완료되었습니다.
